In [1]:
import os
import pandas as pd
from datetime import datetime

# Define constants for file prefixes, extensions, and sheet names
MASTER_FILE_PREFIX = 'Sahamit Report '
MASTER_FILE_EXTENSION = '.xlsx'
SAHAMIT_SHEET_NAME = 'Sahamit Report'
EXCLUDE_SHEET_NAME = 'Exclude'
CLEANED_SHEET_NAME = 'CJ Stock'
PRODUCT_COLUMN = 'Product'
MERGE_INDICATOR_COLUMN = '_from-merged'

# Define file paths as variables
EXCLUDE_CJ_ITEM_PATH = r'D:\Data for Stock Report\Exclude CJ_Item.xlsx'
SOURCE_DIRECTORY = r'T:\SCM Data\Data For Stock\DC_Store'
DESTINATION_DIRECTORY = r'D:\Data for Stock Report'


def get_latest_master_file(directory, file_prefix, file_extension):
    master_files = [f for f in os.listdir(directory) if f.startswith(file_prefix) and f.endswith(file_extension)]
    dates = []
    for file in master_files:
        try:
            date_str = file.replace(file_prefix, '').replace(file_extension, '').strip()
            date_obj = datetime.strptime(date_str, '%d-%m-%Y')
            dates.append((file, date_obj))
        except ValueError:
            continue

    if dates:
        latest_file = max(dates, key=lambda x: x[1])[0]
        return os.path.join(directory, latest_file)
    else:
        return None


def clean_master_file(source_directory, destination_directory):
    latest_file_path = get_latest_master_file(source_directory, MASTER_FILE_PREFIX, MASTER_FILE_EXTENSION)

    if latest_file_path:
        print(f"Loading the latest CJ Stock File: {latest_file_path}")
        master_df = pd.read_excel(latest_file_path, sheet_name=SAHAMIT_SHEET_NAME, header=2)

        # Rename 'Product' column to 'CJ_Item'
        if PRODUCT_COLUMN in master_df.columns:
            master_df.rename(columns={'Product': 'CJ_Item'}, inplace=True)
            print("Column 'Product' has been renamed to 'CJ_Item'")
        else:
            print(f"Column '{PRODUCT_COLUMN}' not found in the DataFrame")

        # Filter out data for 'A-HOME' and 'UNO' divisions
        filter_master_file = ['A-HOME', 'UNO']
        master_df = master_df[~master_df['Division'].isin(filter_master_file)]
        print(f"Filter Division >> {filter_master_file} out of master file")

        # Exclude CJ_Items from a specific Excel file
        print("Loading product list to exclude from this file")
        exclude_df = pd.read_excel(EXCLUDE_CJ_ITEM_PATH, sheet_name=EXCLUDE_SHEET_NAME)

        merged_df = master_df.merge(exclude_df, on='CJ_Item', how='left', indicator=MERGE_INDICATOR_COLUMN)
        master_df = merged_df[merged_df[MERGE_INDICATOR_COLUMN] != 'both']
        master_df.drop(MERGE_INDICATOR_COLUMN, axis=1, inplace=True)

        # Save the cleaned DataFrame to a new Excel file
        cleaned_file_path = os.path.join(destination_directory, 'cleaned_CJ_Stock_Report.xlsx')
        master_df.to_excel(cleaned_file_path, sheet_name=CLEANED_SHEET_NAME, index=False)
        print(f"CJ stock report has been saved to: {cleaned_file_path}")

        return master_df
    else:
        print("Not Found File Stock by CJ.")
        return None


def main():
    clean_master_file(SOURCE_DIRECTORY, DESTINATION_DIRECTORY)


if __name__ == "__main__":
    main()

Loading the latest CJ Stock File: T:\SCM Data\Data For Stock\DC_Store\Sahamit Report 26-05-2025.xlsx
Column 'Product' has been renamed to 'CJ_Item'
Filter Division >> ['A-HOME', 'UNO'] out of master file
Loading product list to exclude from this file


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_18276\3934501441.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_df.drop(MERGE_INDICATOR_COLUMN, axis=1, inplace=True)


CJ stock report has been saved to: D:\Data for Stock Report\cleaned_CJ_Stock_Report.xlsx
